In [1]:
import fibre
import numpy as np
from __future__ import print_function
logger = fibre.utils.Logger(verbose=True)

## 机械臂的SN码可以通过`lsusb`命令查看

In [4]:
teach_arm = fibre.find_any(serial_number="208C31875253", logger=logger)

USB discover loop
ConfigurationValue 1
	InterfaceNumber 0,0
		EndpointAddress 130
	InterfaceNumber 1,0
		EndpointAddress 1
		EndpointAddress 129
	InterfaceNumber 2,0
		EndpointAddress 3
		EndpointAddress 131

Kernel Driver was not attached
EndpointAddress for writing 3
EndpointAddress for reading 131
Connecting to device on USB device bus 0 device 2
JSON: [
{"name":"","id":0,"type":"json","access":"r"},
{"name":"serial_number","id":1,"type":"uint64","access":"r"},
{"name":"get_temperature","id":2,"type":"function","inputs":[],"outputs":[
{"name":"result","id":3,"type":"float","access":"rw"}]},
{"name":"get_voltage","id":4,"type":"function","inputs":[],"outputs":[
{"name":"result","id":5,"type":"float","access":"rw"}]},
{"name":"robot","type":"object","members":[
{"name":"calibrate_home_offset","id":6,"type":"function","inputs":[],"outputs":[]},
{"name":"homing","id":7,"type":"function","inputs":[],"outputs":[]},
{"name":"resting","id":8,"type":"function","inputs":[],"outputs":[]},
{"na

received unexpected ACK: 1009


## RPC连上后,每一个层级都可以直接打印查看

In [5]:
teach_arm

serial_number = 208C31875253 (int)
get_temperature()
get_voltage()
robot:
  calibrate_home_offset()
  homing()
  resting()
  joint_1: ...
  joint_2: ...
  joint_3: ...
  joint_4: ...
  joint_5: ...
  joint_6: ...
  joint_all: ...
  hand: ...
  reboot()
  set_enable(enable: bool)
  set_rgb_enable(enable: bool)
  set_rgb_mode(mode: int)
  move_j(j1: float, j2: float, j3: float, j4: float, j5: float, j6: float)
  move_l(x: float, y: float, z: float, a: float, b: float, c: float)
  set_joint_speed(speed: float)
  set_joint_acc(acc: float)
  set_command_mode(mode: int)
  tuning: ...
  eef_pose: ...

In [6]:
teach_arm.robot

calibrate_home_offset()
homing()
resting()
joint_1:
  angle = 3.8915624618530273 (float)
  velocity = 0.0 (float)
  reboot()
  get_temperature()
  set_enable_temperature(enable: bool)
  erase_configs()
  set_enable(enable: bool)
  set_position_with_time(pos: float, time: float)
  set_position(pos: float)
  set_velocity(vel: float)
  set_velocity_limit(vel: float)
  set_current(current: float)
  set_current_limit(current: float)
  set_node_id(id: int)
  set_acceleration(acc: float)
  apply_home_offset()
  do_calibration()
  set_enable_on_boot(enable: bool)
  set_dce_kp(vel: int)
  set_dce_kv(vel: int)
  set_dce_ki(vel: int)
  set_dce_kd(vel: int)
  set_enable_stall_protect(enable: bool)
  update_angle()
joint_2:
  angle = 0.12867186963558197 (float)
  velocity = 0.0 (float)
  reboot()
  get_temperature()
  set_enable_temperature(enable: bool)
  erase_configs()
  set_enable(enable: bool)
  set_position_with_time(pos: float, time: float)
  set_position(pos: float)
  set_velocity(vel: floa

In [7]:
teach_arm.robot.joint_1

angle = 3.892500162124634 (float)
velocity = 0.0 (float)
reboot()
get_temperature()
set_enable_temperature(enable: bool)
erase_configs()
set_enable(enable: bool)
set_position_with_time(pos: float, time: float)
set_position(pos: float)
set_velocity(vel: float)
set_velocity_limit(vel: float)
set_current(current: float)
set_current_limit(current: float)
set_node_id(id: int)
set_acceleration(acc: float)
apply_home_offset()
do_calibration()
set_enable_on_boot(enable: bool)
set_dce_kp(vel: int)
set_dce_kv(vel: int)
set_dce_ki(vel: int)
set_dce_kd(vel: int)
set_enable_stall_protect(enable: bool)
update_angle()

In [9]:
teach_arm.robot.hand

angle = 38.130001068115234 (float)
get_mode()
get_current_limit()
set_angle(angle: float)
set_mode(mode: int)
set_current_limit(current: float)
set_torque(current: float)
enable_close_loop()
save()

## 接着让我们来测试一些功能

### 机械臂MoveJ

In [10]:
teach_arm.robot.set_enable(True)
teach_arm.robot.move_j(0, 0, 90, 0, 0, 0)

True

### 获取所有关节角度(获取到的这个角度是电机的关节机械角, 和运动学模型角度之间存在一个固定的offset)

In [11]:
joint_offset = np.array([0,-73,180,0,0,0])

In [12]:
all_joint_angles = teach_arm.robot.joint_1.angle, teach_arm.robot.joint_2.angle, teach_arm.robot.joint_3.angle, teach_arm.robot.joint_4.angle, teach_arm.robot.joint_5.angle, teach_arm.robot.joint_6.angle
all_joint_angles = np.round(np.array(all_joint_angles))
all_joint_angles = all_joint_angles + joint_offset
all_joint_angles

array([ 0.,  0., 90.,  0.,  0.,  0.])

### 获取当前末端位姿(旋转使用ZYX欧拉角序列(也称为RPY - Roll, Pitch, Yaw))

In [13]:
def get_current_pose():
    teach_arm.robot.eef_pose.update_pose_6D()
    x,y,z,rz,ry,rx = teach_arm.robot.eef_pose.x, teach_arm.robot.eef_pose.y, teach_arm.robot.eef_pose.z, teach_arm.robot.eef_pose.a, teach_arm.robot.eef_pose.b, teach_arm.robot.eef_pose.c
    current_pose = np.array([x,y,z,rz,ry,rx])
    return current_pose

current_pose = get_current_pose()
current_pose


array([227.50428772,   0.        , 324.49487305,   0.        ,
        90.        ,   0.        ])

### 机械臂MoveL(直接控制末端在笛卡尔空间运动, 输入单位是mm)

In [14]:
# 沿着x轴方向移动10mm
teach_arm.robot.move_l(current_pose[0]+10, current_pose[1], current_pose[2], current_pose[3], current_pose[4], current_pose[5])
current_pose = get_current_pose()
current_pose

array([227.49372864,   0.        , 324.50512695,   0.        ,
        90.        ,   0.        ])

In [15]:
# 沿着z轴方向移动10mm
teach_arm.robot.move_l(current_pose[0], current_pose[1], current_pose[2]+10, current_pose[3], current_pose[4], current_pose[5])
current_pose = get_current_pose()
current_pose


array([237.49632263,   0.        , 324.16833496,   0.        ,
        90.        ,   0.        ])

### 归巢

In [16]:
teach_arm.robot.resting()

### 复位回到home pose

In [17]:
teach_arm.robot.move_j(0, 0, 90, 0, 0, 0)

True

### 示教臂的夹爪实时位置读取

In [18]:
## 让夹爪电机进入零力矩状态
teach_arm.robot.hand.set_mode(0)
teach_arm.robot.hand.set_torque(0)

In [19]:
## 拖动夹爪然后查看角度变化
rate = 0.02  # 50Hz
import time
while True:
    start_time = time.time()
    print(teach_arm.robot.hand.angle)

38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931884766
38.119998931

KeyboardInterrupt: 

### 示教臂的夹爪实时力矩读取(即将支持,硬件施工中)

### Follow Arm相机测试

In [20]:
import cv2
mjpg_streamer_url = "http://192.168.65.110:8080/?action=stream"
cap = cv2.VideoCapture(mjpg_streamer_url)
while True:
    ret, frame = cap.read()
    cv2.imshow("frame", frame)
    cv2.waitKey(1)
cv2.destroyAllWindows()

2025-03-21 15:16:01.361 python[63765:10187927] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-21 15:16:01.361 python[63765:10187927] +[IMKInputSession subclass]: chose IMKInputSession_Modern


KeyboardInterrupt: 

error in usbbulk_transport.py, process_packet
Traceback (most recent call last):
  File "/Users/yinzi/Downloads/Dummy_V2_workspace/dummy_ai/dummy_ctrl/fibre/usbbulk_transport.py", line 127, in get_packet
    ret = self.epr.read(bufferLen, timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yinzi/Downloads/Dummy_V2_workspace/dummy_ai/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/core.py", line 431, in read
    return self.device.read(self, size_or_buffer, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yinzi/Downloads/Dummy_V2_workspace/dummy_ai/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/core.py", line 1043, in read
    ret = fn(
          ^^^
  File "/Users/yinzi/Downloads/Dummy_V2_workspace/dummy_ai/dummy_ctrl/.venv/lib/python3.11/site-packages/usb/backend/libusb1.py", line 850, in bulk_read
    return self.__read(self.lib.libusb_bulk_transfer,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/yinzi/Downl